In [1]:
%pip install faiss-cpu langchain sentence_transformers rank-bm25 kiwipiepy
%pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import glob

def split_text_into_chunks(
    text: str,
    chunk_size: int = 512,
    overlap: int = 100
) -> list:
    """
    text를 chunk_size씩 잘라서 리스트로 반환.
    각 청크 사이에 overlap 길이만큼 문자가 겹치도록 함.

    예) chunk_size=512, overlap=100 => 실제 step=412
    즉,
      - 첫 청크: [0:512]
      - 두 번째 청크: [412:412+512] (앞 100자가 겹침)
      - 세 번째 청크: [824:824+512]
      ...
    """
    chunks = []
    step = chunk_size - overlap
    if step <= 0:
        raise ValueError("chunk_size must be greater than overlap")

    text_length = len(text)
    start = 0

    while start < text_length:
        end = min(start + chunk_size, text_length)
        chunk = text[start:end]
        chunks.append(chunk)

        start += step  # 겹치기 고려하여 다음 청크 시작점 이동

    return chunks


def load_docs_from_txt(directory: str, chunk_size=1024, overlap=200):
    """
    지정한 디렉토리에 있는 모든 txt 파일을 읽은 후,
    chunk_size, overlap으로 청크를 분할하여
    [{'doc_id': ..., 'file_name': ..., 'chunk_index': ..., 'chunk_text': ...}, ...] 형태 리스트를 반환
    """
    docs = []
    doc_id = 0

    for file_path in glob.glob(os.path.join(directory, '*.txt')):
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read().strip()

        chunks = split_text_into_chunks(text, chunk_size, overlap)

        for i, chunk_text in enumerate(chunks):
            docs.append({
                "doc_id": doc_id,
                "file_name": os.path.basename(file_path),
                "chunk_index": i,
                "chunk_text": chunk_text
            })
        doc_id += 1

    return docs

# load_docs_from_txt('/content/drive/MyDrive/BITAmin/컨퍼런스/data')


In [3]:
from rank_bm25 import BM25Okapi

def build_bm25_index(docs):
    """
    docs 목록을 받아서 BM25 인덱스(BM25Okapi) 객체와
    문서 정보(문서 리스트)를 반환
    """
    # docs: [{'doc_id':..., 'chunk_text':...}, ...]

    # BM25Okapi에 넣을 때는 토큰화가 필요
    # 간단히 whitespace split(또는 konlpy, mecab 등 사용 가능)
    tokenized_corpus = []
    for d in docs:
        tokens = d["chunk_text"].split()
        tokenized_corpus.append(tokens)

    bm25 = BM25Okapi(tokenized_corpus)
    return bm25


In [4]:
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

def build_faiss_index(docs):
    """
    주어진 문서(docs)를 LangChain Document로 변환 후
    Faiss 인덱스를 생성 & 반환
    """
    # 1) 임베딩 모델 로드 (E5 모델 예시)
    embedding_model = "intfloat/multilingual-e5-large"
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

    # 2) langchain Document로 변환
    #    => chunk_text를 page_content로, 나머지를 metadata로
    langchain_docs = []
    for d in docs:
        langchain_docs.append(Document(
            page_content=d["chunk_text"],
            metadata={
                "doc_id": d["doc_id"],
                "file_name": d["file_name"],
                "chunk_index": d["chunk_index"]
            }
        ))

    # 3) FAISS VectorStore 생성
    faiss_store = FAISS.from_documents(langchain_docs, embeddings)

    # (옵션) 로컬에 인덱스 저장
    # faiss_store.save_local("faiss_index")

    return faiss_store


# reranker 적용

In [6]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def exp_normalize(x):
    """
    logits 벡터 x를 지수 정규화(softmax 유사)하여 0~1 범위 점수로 변환
    """
    b = x.max()
    y = np.exp(x - b)
    return y / y.sum()

def rerank_with_cross_encoder(query, retrieved_docs, model, tokenizer, top_k=5):
    """
    1차 검색 결과(retrieved_docs)에 대해,
    (query, doc["chunk_text"]) 쌍으로 CrossEncoder를 통해 점수를 다시 계산 후
    최종 top_k 문서를 Re-ranking하여 반환

    - query: str
    - retrieved_docs: [{'doc_id':..., 'chunk_text':..., ...}, ...]
    - model: AutoModelForSequenceClassification (CrossEncoder)
    - tokenizer: 해당 모델에 맞는 토크나이저
    - top_k: 최종 몇 개를 반환할지
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    # (query, chunk_text) 형태의 인풋 쌍 생성
    pairs = [(query, d["chunk_text"]) for d in retrieved_docs]

    # 토크나이징
    inputs = tokenizer(
        pairs,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

    # 텐서를 GPU/CPU에 올림
    for k, v in inputs.items():
        inputs[k] = v.to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        # 모델에 따라 logits shape가 [batch, 1] or [batch, 2] 등 다양할 수 있음
        # 여기서는 [batch] 형태로 가정
        logits = outputs.logits.view(-1).float()  # shape: [batch_size]

    # logits 자체를 점수로 사용하거나, 필요하면 softmax/exp_normalize 적용
    scores = logits.cpu().numpy()
    # scores = exp_normalize(scores)  # 배치 내에서만 확률분포를 보고 싶다면

    # 점수에 따라 정렬
    doc_score_pairs = list(zip(retrieved_docs, scores))
    doc_score_pairs.sort(key=lambda x: x[1], reverse=True)

    # 상위 top_k
    reranked_docs = [doc for doc, s in doc_score_pairs[:top_k]]
    return reranked_docs

def hybrid_search_with_reranker(
    query: str,
    faiss_store: FAISS,
    bm25,
    docs,
    embeddings: HuggingFaceEmbeddings,
    alpha: float = 0.5,
    top_k: int = 5,
    reranker_model=None,
    reranker_tokenizer=None
):
    """
    1) 기존 Hybrid Search 로직으로 상위 top_k*5 (or 50)만큼 문서 후보를 얻음
    2) Reranker(CrossEncoder)가 있으면 그 후보들에 대해 Re-ranking
       => 최종 top_k 문서 반환
    """

    # =============== 1단계: Hybrid Search ===============
    import numpy as np

    # 1) query 임베딩
    query_embedding = embeddings.embed_query(query)
    query_embedding = np.array(query_embedding, dtype=np.float32)

    # 2) Faiss 검색
    N = max(top_k*5, 50)  # 2차 랭킹용 후보를 넉넉히 확보
    dense_results = faiss_store.similarity_search_with_score(query, k=N)
    # dense_results: List[ (Document, float(similarity)) ]

    # 3) BM25 점수
    from rank_bm25 import BM25Okapi
    query_tokens = query.split()
    bm25_scores = bm25.get_scores(query_tokens)  # length=len(docs)

    # 4) dense_results -> doc2dense (unique_key로 매핑)
    doc2dense = {}
    for (doc_obj, score) in dense_results:
        md = doc_obj.metadata
        d_id = md["doc_id"]
        c_idx = md["chunk_index"]
        unique_key = d_id * 1_000_000 + c_idx
        doc2dense[unique_key] = score

    # 5) final_score = alpha*dense + (1-alpha)*sparse
    results = []
    for i, d in enumerate(docs):
        d_id = d["doc_id"]
        c_idx = d["chunk_index"]
        unique_key = d_id * 1_000_000 + c_idx

        dense_s = doc2dense.get(unique_key, 0.0)
        sparse_s = bm25_scores[i]
        final_s = alpha*dense_s + (1-alpha)*sparse_s
        results.append((final_s, i))

    # 상위 N개 선별 (N = top_k*5)
    results.sort(key=lambda x: x[0], reverse=True)
    top_indices = [idx for (_, idx) in results[:N]]
    first_pass_docs = [docs[i] for i in top_indices]

    # =============== 2단계: Re-ranking (옵션) ===============
    if (reranker_model is not None) and (reranker_tokenizer is not None):
        # 2차 랭킹을 통해 최종 top_k 뽑기
        reranked_docs = rerank_with_cross_encoder(
            query=query,
            retrieved_docs=first_pass_docs,
            model=reranker_model,
            tokenizer=reranker_tokenizer,
            top_k=top_k
        )
        return reranked_docs
    else:
        # Reranker 미지정 시, 그냥 1단계 결과 중 top_k만
        return first_pass_docs[:top_k]


In [ ]:
# 기존 코드: docs, faiss_store, bm25, embeddings 준비
docs_directory = "/data/ephemeral/home/gyeom/data"
docs = load_docs_from_txt(docs_directory, chunk_size=300, overlap=40)
faiss_store = build_faiss_index(docs)
bm25 = build_bm25_index(docs)
faiss_store.save_local("faiss_index")
# bm25.save_local("bm25_index")

embedding_model = "intfloat/multilingual-e5-large"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

# 1) Reranker 모델 (예: ko-reranker)
from transformers import AutoTokenizer, AutoModelForSequenceClassification
reranker_name = "Dongjin-kr/ko-reranker"  # 예시 모델
reranker_tokenizer = AutoTokenizer.from_pretrained(reranker_name)
reranker_model = AutoModelForSequenceClassification.from_pretrained(reranker_name)



In [ ]:
# 2) Hybrid + Reranker
query = "동학 농민 운동"
final_docs = hybrid_search_with_reranker(
    query=query,
    faiss_store=faiss_store,
    bm25=bm25,
    docs=docs,
    embeddings=embeddings,
    alpha=0.5,
    top_k=15,
    reranker_model=reranker_model,
    reranker_tokenizer=reranker_tokenizer
)

def build_prompt(docs, query):
    """
    docs: List of dict, each dict has 'chunk_text'.
    user_query: str, 예) "동학 농민 운동과 신분제 폐지의 배경"
    => 적절히 연결한 Prompt 문자열을 구성해 반환
    """
    context_text = ""
    for i, d in enumerate(docs):
        context_text += f"\n[문서 {i}]\n{d['chunk_text']}\n"

    return context_text



prompt_text = build_prompt(final_docs, query)
print(prompt_text)



[문서 0]
각종 배상금 지불, 일본의 경제적 침투 등으로 농민층의 불안과 불만은 팽배하였다. 정치⋅사회적 의식이 급성장한 농촌 지식인과 농민의 사회 변혁 욕구도 높아졌다. 이러한 분위기 속에서 인간 평등과 사회 개혁을 주장한 동학이 삼남지방을 중심으로 확산되었다.

동학 농민 운동은 1894년 전라도 고부에서 시작되었다. 전봉준을 중심으로 한 농민층은 고부 군수 조병갑의 탐욕스럽고 포악함에 봉기한 이후, 보국안민과 제폭구민을 내세우며 전라도 일대를 장악하였다. 정부와 농민군은 전주에서 폐단이 많은 정치를 개혁하기로 합의하였다. 특히 농민군은 

[문서 1]
이 많은 정치를 개혁하기로 합의하였다. 특히 농민군은 각지에 집강소를 설치하여 이를 실천해 나갔다. 그러나 일본군이 청⋅일 전쟁을 일으키면서 내정을 간섭하자, 농민군은 다시 봉기하여 외세를 몰아 내기 위하여 서울로 진격하였다. 하지만, 톈진 조약을 빙자하여 우리 나라에 파견된 우세한 무기로 무장한 일본군에게 농민군은 공주 우금치에서 패하고, 지도부가 체포되면서 이 운동은 실패로 끝났다.

동학 농민 운동은 농민층이 전통적 지배 체제에 반대하는 개혁 정치를 요구하고, 외세의 침략을 자주적으로 물리치려 했다는 점에서 아래로부터의 반봉건

[문서 2]
 받아들여 평등한 근대 사회를 만들려고 하였다. 급진 개화파 세력은 1884년에 갑신정변을 일으켜 문벌을 없애고 능력에 따라 인재를 고루 등용하려 했으며, 인민 평등의 권리를 선언하는 등 사회의 전반적인 근대화를 추진하고자 하였다.

1860년대에 등장한 동학은 “사람 섬기기를 하늘같이 하라.”라고 하여, 사람은 누구나 평등하다는 사상을 가지고 있었다. 이러한 평등 사상에 기초한 동학은 민중 속으로 빠르게 퍼져 나갔고, 1894년 동학 농민 운동으로 발전하였다. 동학 농민 운동은 일본군과 조선 관군의 진압으로 좌절되었지만, 양반 중

[문서 3]
 자주적으로 물리치려 했다는 점에서 아래로부터의 반봉건적, 반침략적 민족 운동이었다. 비록 당시의 집권 세력과 일본 

# reranker 적용 안함

In [5]:
import numpy as np
from heapq import nlargest

def hybrid_search(
    query: str,
    faiss_store: FAISS,
    bm25,
    docs,
    embeddings: HuggingFaceEmbeddings,
    alpha: float = 0.5,
    top_k: int = 5
):
    """
    query에 대해:
      1) Faiss에서 top-N (~= top_k * 5 정도) 문서를 찾고, dense 점수(= 코사인 유사도) 계산
      2) BM25 점수(스파스)를 docs 전체에 대해 계산
      3) 두 점수를 가중합(alpha)으로 결합해 상위 top_k 문서를 최종 선정

    - alpha: dense 점수와 BM25 점수를 어떻게 섞을지 결정 (0.0 ~ 1.0)
    - top_k: 최종적으로 반환할 문서 개수
    """

    # 1) query 임베딩 구하기
    query_embedding = embeddings.embed_query(query)
    query_embedding = np.array(query_embedding, dtype=np.float32)

    # 2) Faiss 쪽 검색
    #  - langchain >= 0.0.164 버전에는 `similarity_search_with_score()`
    #    만약 지원 안 하면, 일단 top-N개의 문서를 가져와서 재계산.
    #    아래 예시로 top 50개 문서를 대상으로 하이브리드 처리
    N = max(top_k*5, 50)

    dense_results = faiss_store.similarity_search_with_score(query, k=N)
    # dense_results: List[ (Document, float(similarity)) ]
    # similarity: cos 유사도 (1.0에 가까울수록 유사) - 버전에 따라 다를 수 있음

    # 3) BM25 쪽 검색
    #    - docs 전체에 대해 query의 BM25 점수를 구한다
    from rank_bm25 import BM25Okapi
    query_tokens = query.split()
    bm25_scores = bm25.get_scores(query_tokens)  # length = len(docs)

    # 4) dense_results를 doc_id 기준으로 매핑 { index_in_docs : dense_score }
    doc2dense = {}
    # docs 리스트와 dense_results가 "어떤 index"로 연결되어 있는지 알아야 함
    # 보통 doc.metadata["doc_id"], doc.metadata["chunk_index"] 등을 사용
    for (doc_obj, score) in dense_results:
        md = doc_obj.metadata
        d_id = md["doc_id"]
        c_idx = md["chunk_index"]
        # docs 리스트에서 (doc_id, chunk_index)가 어느 인덱스인지 찾는 로직 필요
        # 예: "key = (doc_id, chunk_index)"
        # 이걸 docs에서 검색하거나, 미리 build 해둔 인덱스(lookup)로 찾을 수 있음
        # 아래는 간단히 docs를 loop 돌며 찾을 수도 있음(비효율적).
        # -> 실제로는 dict[(doc_id, chunk_index)] = i 식으로 미리 만들어두면 좋음

        # 간단히 "d_id * 1_000_000 + c_idx"를 key로:
        unique_key = d_id * 1_000_000 + c_idx
        doc2dense[unique_key] = score

    # 5) 최종 점수를 합산(hybrid)하기
    #   - docs[i]에 대해서:
    #       BM25 점수 = bm25_scores[i]
    #       Dense 점수 = doc2dense.get(unique_key, 0)
    #     => final_score = alpha * dense + (1-alpha) * bm25
    #   - BM25와 Dense 스케일이 다를 수 있으므로, 실제로는 min-max 정규화나,
    #     z-score 표준화가 필요할 수도 있음 (여기서는 단순 예시)

    results = []
    for i, d in enumerate(docs):
        d_id = d["doc_id"]
        c_idx = d["chunk_index"]
        unique_key = d_id * 1_000_000 + c_idx

        dense_s = doc2dense.get(unique_key, 0.0)      # 없으면 0
        sparse_s = bm25_scores[i]                    # BM25 점수
        final_s = alpha*dense_s + (1-alpha)*sparse_s

        results.append((final_s, i))

    # 6) 최종 상위 top_k 선별
    results.sort(key=lambda x: x[0], reverse=True)
    top_indices = [idx for (_, idx) in results[:top_k]]

    # 7) 반환용
    out = []
    for ti in top_indices:
        out.append(docs[ti])  # docs[ti] 구조: {'doc_id','chunk_index','chunk_text','file_name',...}
    return out


In [6]:

docs_directory = "/data/ephemeral/home/gyeom/data/data"

docs = load_docs_from_txt(docs_directory, chunk_size=700, overlap=50)

# FAISS 인덱스
faiss_store = build_faiss_index(docs)
# BM25 인덱스
bm25 = build_bm25_index(docs)

# 같은 임베딩 객체를 하이브리드로도 사용
embedding_model = "intfloat/multilingual-e5-large"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)



/tmp/ipykernel_55317/395744265.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# 2) Hybrid + Reranker
query = "세종대왕의 훈민정음 창제 스토리"
final_docs = hybrid_search(
    query=query,
    faiss_store=faiss_store,
    bm25=bm25,
    docs=docs,
    embeddings=embeddings,
    alpha=0.35,
    top_k=4
)

def build_prompt(docs, query):
    """
    docs: List of dict, each dict has 'chunk_text'.
    user_query: str, 예) "동학 농민 운동과 신분제 폐지의 배경"
    => 적절히 연결한 Prompt 문자열을 구성해 반환
    """
    context_text = ""
    for i, d in enumerate(docs):
        context_text += f"\n[문서 {i}]\n{d['chunk_text']}\n"

    return context_text



prompt_text = build_prompt(final_docs, query)
print(prompt_text)



[문서 0]
. 

인간 세종-육식남 세종 
서울 광화문 세종로에는 세종대왕 동상이 있지요. 왼손에는 책을 들고 오른손은 인자하게 백성들을 향해 뻗고 있는 모습입니다. 하지만 진짜 세종대왕의 모습은 조금 달랐을 것 같네요. 왜냐하면 세종대왕은 실제로 살집이 많 은 걸 넘어 육중한 몸매의 소유자였거든요. 온라인에서 제공하는 「조선왕조실록」에서 고기반찬을 뜻하는 '육선'이란 단어를 검색하면 세종대왕 시절의 일화가 압도적으로 많다는 걸알수 있어요. 원래 세종의 집안은 할아버지인 태조와 아버지 태종에서 알 수 있듯 무인 집안입니다. 따라서 다들 체격이 좋고, 숙련된 말타기와 활 솜씨는 기본 소양으로 갖고 있었지요. 이런 모습을 똑 닮은 사람이 있으니, 바로 세종의 제일 큰 형이자, 왕이 될 뻔한 양녕이에요. 하지만 세종은 달랐어요. 활쏘기나 말타기 대신 고기와 책을 가까이했기 때 문이지요. 삼시 세끼 육식을 즐기며 온종일 방안에서 책만 보면 사람이 어떻게 될까요? 당연히 뚱뚱해질 수밖에 없어요. 물 론 조선의 왕들이 대부분 비만인 건 맞지만, 그래도 엄연한 무인집안 자식인데 살이 쪄도 너무 찌니, 아버지 태종 의 걱정이 늘다 못해 다음과 같이 말합니다. "주상은 사냥을 좋아하지 않으시나, 몸이 비중(문고)하시니 마땅히 때때로 나와 노니셔서 몸을 존절히 하셔야 하겠으며, 또 문과 무에 어느 하나를 편벽되어 폐할 수는 없은즉, 나는 장차 주상과 더불어 무사

[문서 1]
 내비칩니다. 하지만 마흔 살이 되던 1436년에 왕세자의 대리청정 문제를 꺼냈으나, 신하들이 반대하는 바람에 뜻을 굽히다 이후 1442년에 이르러 결국 뜻을 이룹니다. 그러나 세종은 왕세자에게 많은 업무를 이양한 뒤에도 여전히 격무를 수행합니다. 오히려 몸이 아플 때, 더욱 놀 라운 업적을 쌓으니, 그게 바로 '훈민정음' 창제입니다. 
1443년 한글의 창조자 세종 
전 세계 유일무이 창시자와 반포일, 창제 원리까지 후손들이 모두 잘 알고 있는, 바로 자랑스러운 우리 문화, 한글을 창제한 것이지요. 

# llm 연결

In [8]:
# -*- coding: utf-8 -*-

import requests


class CompletionExecutor:
    def __init__(self, host, api_key, request_id):
        self._host = host
        self._api_key = api_key
        self._request_id = request_id

    def execute(self, completion_request):
        headers = {
            'Authorization': self._api_key,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id,
            'Content-Type': 'application/json; charset=utf-8',
            'Accept': 'text/event-stream'
        }

        # event:result에 해당하는 data만 저장할 리스트
        store_data = []

        with requests.post(
            self._host + '/testapp/v1/chat-completions/HCX-DASH-001',
            headers=headers,
            json=completion_request,
            stream=True
        ) as r:
            is_result_event = False  # 직전 라인이 event:result인지 판별하기 위한 플래그

            for line in r.iter_lines():
                if line:
                    decoded_line = line.decode("utf-8").strip()

                    # event:result 라인이 나오면 플래그 설정
                    if decoded_line.startswith("event:result"):
                        is_result_event = True

                    # event:result 바로 다음에 오는 data 라인일 경우 데이터를 추출하여 저장
                    elif decoded_line.startswith("data:") and is_result_event:
                        data_content = decoded_line[len("data:"):].strip()
                        store_data.append(data_content)
                        is_result_event = False

        # 여기서는 event:result로 표시된 data만 store_data에 담겨있습니다.
        # 필요에 따라 반환 혹은 다른 처리 가능
        return store_data


In [10]:

if __name__ == '__main__':
    # 예시 사용
    completion_executor = CompletionExecutor(
        host='https://clovastudio.stream.ntruss.com',
        api_key='Bearer nv-d402ca8957604eb1aa8656415b89101cazC4',
        request_id='6dc56592f44747789164dc55eb86227a'
    )

    preset_text = [
        {
            "role": "system",
            "content": f"""당신은 조선왕조실록 전문가입니다. 아래에 제공된 문서들의 구체적 내용을 바탕으로, {query}의 정답을 단답형식으로 말해주세요.


            """
        },
        {
            "role": "user",
            "content": f'''{prompt_text}의 내용을 토대로 {query}의 정답을 단답형식으로 말하세요

            답변:'''
        }
    ]

    request_data = {
        'messages': preset_text,
        'topP': 0.8,
        'topK': 0,
        'maxTokens': 600,
        'temperature': 0.5,
        'repeatPenalty': 5.0,
        'stopBefore': [],
        'includeAiFilters': True,
        'seed': 0
    }

    # print(preset_text)
    result = completion_executor.execute(request_data)
    result

In [12]:
%pip install pandas

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/44/50/7db2cd5e6373ae796f0ddad3675268c8d59fb6076e66f0c339d61cea886b/pandas-2.2.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 457.7 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/0f/dd/84f10e23edd882c6f968c21c2434fe67bd4a528967067515feca9e611e5e/tzdata-2025.1-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 18.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.8/346.8 kB 48.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import json
import time
from tqdm import tqdm  # tqdm 임포트

# 가정: 아래 객체들은 이미 정의되어 있다고 가정
# faiss_store, bm25, docs, embeddings, hybrid_search_with_reranker, reranker_model, reranker_tokenizer
# build_prompt, CompletionExecutor

def run_rag_and_llm(df):
    # 1) CompletionExecutor 인스턴스 생성 예시
    completion_executor = CompletionExecutor(
        host='https://clovastudio.stream.ntruss.com',
        api_key='Bearer nv-d402ca8957604eb1aa8656415b89101cazC4',
        request_id='6dc56592f44747789164dc55eb86227a'
    )

    df['LLM답변'] = None

    # 3) df의 각 행을 순회 (tqdm로 진행상황 표시)
    for idx in tqdm(range(len(df)), desc="Processing rows"):
        # 간혹 너무 빠르게 API를 호출하면 제한이 걸릴 수 있으므로 잠시 대기
        time.sleep(3)

        # (a) 현재 질문 꺼내기
        query = df['질문'].iloc[idx]
        ans = df['답변'].iloc[idx]
        llmans = df['LLM답변'].iloc[idx]

        # (b) RAG 검색(하이브리드 + reranker) 수행
        final_docs = hybrid_search(
        query = query,
        faiss_store=faiss_store,
        bm25=bm25,
        docs = docs,
        embeddings=embeddings,
        alpha=0.35,
        top_k=5)

        # (c) 검색결과 기반 프롬프트 생성
        prompt_text = build_prompt(final_docs, query)

        # (d) 프롬프트에 맞춰 system/user 메시지 작성
        preset_text = [
            {
                "role": "system",
                "content": f"""당신은 조선왕조실록 전문가입니다. 아래에 제공된 문서들의 구체적 내용을 바탕으로, {query}의 정답을 단답형식으로 말해주세요.


                """
            },
            {
                "role": "user",
                "content": f'''{prompt_text}의 내용을 토대로 {query}의 정답을 단답형식으로 말하세요

                답변:'''
            }
        ]

        request_data = {
            'messages': preset_text,
            'topP': 0.8,
            'topK': 0,
            'maxTokens': 600,
            'temperature': 0.5,
            'repeatPenalty': 5.0,
            'stopBefore': [],
            'includeAiFilters': True,
            'seed': 0
        }

        # (e) LLM API 호출
        result = completion_executor.execute(request_data)

        # (f) SSE로부터 받은 응답(JSON) 파싱 (streamed data 중 첫 번째만 취급)
        if len(result) > 0:
            json_str = result[0]
            parsed_response = json.loads(json_str)
            assistant_content = parsed_response["message"]["content"]
        else:
            assistant_content = ""  # 혹은 "응답없음"

        # (g) DataFrame에 결과 저장
        df.at[idx, 'LLM답변'] = assistant_content

    return df



In [10]:
import pandas as pd

def txt_to_df_qa(txt_path, output_csv=None):
    """
    txt 파일에
       질문: ...
       답변: ...
    형태로 기록된 라인을 파싱하여
    DataFrame(질문, 답변) 형태로 반환.
    (optionally CSV로 저장)
    """

    with open(txt_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    questions = []
    answers = []
    current_q = None
    current_a = None

    for line in lines:
        line = line.strip()
        if line.startswith("질문:"):
            # 질문: 뒤 텍스트만 추출
            current_q = line.replace("질문:", "").strip()
        elif line.startswith("답변:"):
            # 답변: 뒤 텍스트만 추출
            current_a = line.replace("답변:", "").strip()
            # 질문-답변 한 쌍을 리스트에 저장
            questions.append(current_q)
            answers.append(current_a)
            # 다음 쌍을 위해 초기화
            current_q = None
            current_a = None
        else:
            # 빈 줄 or 기타 텍스트는 무시
            pass

    # DataFrame 생성
    df = pd.DataFrame({
        '질문': questions,
        '답변': answers
    })

    # 필요 시 CSV 저장
    if output_csv:
        df.to_csv(output_csv, index=False, encoding='utf-8-sig')

    return df


# 사용 예시
if __name__ == "__main__":
    txt_file_path = "/data/ephemeral/home/gyeom/evaldata_fffffff.txt"
    df_qa = txt_to_df_qa(txt_file_path, output_csv="myQA.csv")


In [11]:
ori = run_rag_and_llm(df_qa)

Processing rows: 100%|██████████| 100/100 [06:54<00:00,  4.15s/it]


In [12]:
ori

,질문,답변,LLM답변
0,조선을 건국한 왕은 누구인가요?,태조,태조 이성계
1,태조가 고려에 등 돌린 결정적 사건은 무엇인가요?,위화도 회군,
2,조선 3대 왕은 누구인가요?,태종,태종 이방원
3,사병을 혁파한 왕은 누구인가요?,태종,태종 이방원
4,한양 천도를 주도한 왕은 누구인가요?,태조,정도전
...,...,...,...
95,홍경래의 난(1811)이 일어난 시기의 왕은 누구인가요?,순조,순조
96,"1801년 신유박해 외에, 1815년에도 천주교 관련 옥사가 일어난 이유는 무엇인가요?",사교 탄압,신유박해 이후에도 천주교 신자들이 계속해서 발견되었기 때문입니다.
97,철종 시기에 발생한 임술농민봉기(1862)는 주로 무엇에 대한 반발이었나요?,과도한 세금,삼정의 문란에 대한 반발
98,고종 즉위 직후 흥선대원군이 비변사를 폐지한 목적은 무엇이었나요?,왕권 강화,왕권 강화 \n\n고종 즉위 직후 흥선대원군은 비변사를 폐지하고 의정부와 삼군부를...


In [13]:
import pandas as pd
import json
import time
from tqdm import tqdm  # tqdm 임포트

# 가정: 아래 객체들은 이미 정의되어 있다고 가정
# faiss_store, bm25, docs, embeddings, hybrid_search_with_reranker, reranker_model, reranker_tokenizer
# build_prompt, CompletionExecutor

def run_rag_and_llm(df):
    # 1) CompletionExecutor 인스턴스 생성 예시
    completion_executor = CompletionExecutor(
        host='https://clovastudio.stream.ntruss.com',
        api_key='Bearer nv-d402ca8957604eb1aa8656415b89101cazC4',
        request_id='6dc56592f44747789164dc55eb86227a'
    )

    df['채점결과'] = None

    # 3) df의 각 행을 순회 (tqdm로 진행상황 표시)
    for idx in tqdm(range(len(df)), desc="Processing rows"):
        # 간혹 너무 빠르게 API를 호출하면 제한이 걸릴 수 있으므로 잠시 대기
        time.sleep(3)

        # (a) 현재 질문 꺼내기
        query = df['질문'].iloc[idx]
        ans = df['답변'].iloc[idx]
        llmans = df['LLM답변'].iloc[idx]

        # (b) RAG 검색(하이브리드 + reranker) 수행
        # final_docs = hybrid_search(
        # query = query,
        # faiss_store=faiss_store,
        # bm25=bm25,
        # docs = docs,
        # embeddings=embeddings,
        # alpha=0.5,
        # top_k=15)

        # (c) 검색결과 기반 프롬프트 생성
        # prompt_text = build_prompt(final_docs, query)

        # (d) 프롬프트에 맞춰 system/user 메시지 작성
        preset_text = [
            {
                "role": "system",
                "content": """
                당신은 한국사 채점 전문가입니다.
                다음 채점 기준을 엄격하게 적용하여 수험자의 답변을 평가하세요.

                **채점 기준 (1~5점):**
                - ✅ **5점**: 정답과 완전히 일치하는 경우 (예: 정답 '태조' → 답변 '태조').
                - ✅ **4점**: 정답을 포함하지만 추가 설명이 있는 경우 (예: 정답 '태조' → 답변 '태조 이성계').
                - ⚠️ **3점**: 유사하지만 정답이 애매한 경우 (예: 정답 '태조' → 답변 '조선 태조').
                - ⚠️ **2점**: 정답과 다소 차이가 있는 경우 (예: 정답 '태조' → 답변 '세종대왕').
                - ❌ **1점**: 완전히 틀린 답변 또는 관련 없는 내용 (예: 정답 '태조' → 답변 '고려 시대 왕').

                **최종 판정 방식:**
                - 4점 이상이면 `1`을 출력하세요.
                - 3점 이하이면 `0`을 출력하세요.

                **예시**
                질문: "조선을 건국한 왕의 이름은?"
                정답: '태조'
                수험자의 답변: '태조 이성계'
                채점 결과: 4점 → 출력값: 1

                질문: "조선을 건국한 왕의 이름은?"
                정답: '태조'
                수험자의 답변: '세종대왕'
                채점 결과: 2점 → 출력값: 0

                이 채점 기준을 엄격하게 적용하세요.

                """
            },
            {
                "role": "user",
                "content": (
                    f"질문: '{query}'\n"
                    f"정답: '{ans}'\n"
                    f"수험자의 답변(LLM답변): '{llmans}'\n\n"
                    "채점해주세요."
                    "답변 : "
                )
            }
        ]

        request_data = {
            'messages': preset_text,
            'topP': 0.8,
            'topK': 0,
            'maxTokens': 600,
            'temperature': 0.5,
            'repeatPenalty': 5.0,
            'stopBefore': [],
            'includeAiFilters': True,
            'seed': 0
        }

        # (e) LLM API 호출
        result = completion_executor.execute(request_data)

        # (f) SSE로부터 받은 응답(JSON) 파싱 (streamed data 중 첫 번째만 취급)
        if len(result) > 0:
            json_str = result[0]
            parsed_response = json.loads(json_str)
            assistant_content = parsed_response["message"]["content"]
        else:
            assistant_content = ""  # 혹은 "응답없음"

        # (g) DataFrame에 결과 저장
        df.at[idx, '채점결과'] = assistant_content

    return df



In [14]:
ori = run_rag_and_llm(ori)

Processing rows: 100%|██████████| 100/100 [06:58<00:00,  4.19s/it]


In [15]:
ori

,질문,답변,LLM답변,채점결과
0,조선을 건국한 왕은 누구인가요?,태조,태조 이성계,채점 결과 : 4점\n출력값 : 1 \n\n이유 : '태조 이성계'는 '태조'를 포...
1,태조가 고려에 등 돌린 결정적 사건은 무엇인가요?,위화도 회군,,"⚠️ 2점\n\n정답인 '위화도 회군'을 언급하지 않았습니다. \n\n추가로, 해..."
2,조선 3대 왕은 누구인가요?,태종,태종 이방원,"채점 결과 : 4점\n\n정답인 '태종'을 포함하면서, 추가 설명이 있으므로 4점을..."
3,사병을 혁파한 왕은 누구인가요?,태종,태종 이방원,"채점 결과 : 4점\n\n정답인 '태종'을 포함하면서, 해당 인물의 출신과 업적을 ..."
4,한양 천도를 주도한 왕은 누구인가요?,태조,정도전,"채점 결과 : 2점\n\n정답인 '태조'가 아닌 '정도전'으로 답변하였으므로, 점수..."
...,...,...,...,...
95,홍경래의 난(1811)이 일어난 시기의 왕은 누구인가요?,순조,순조,채점 결과 : 5점 \n\n정답과 완전히 일치하는 답변입니다.
96,"1801년 신유박해 외에, 1815년에도 천주교 관련 옥사가 일어난 이유는 무엇인가요?",사교 탄압,신유박해 이후에도 천주교 신자들이 계속해서 발견되었기 때문입니다.,채점 결과 : 4점\n\n정답인 '사교 탄압'을 포함하면서도 추가적인 설명을 잘 해...
97,철종 시기에 발생한 임술농민봉기(1862)는 주로 무엇에 대한 반발이었나요?,과도한 세금,삼정의 문란에 대한 반발,채점 결과 : 4점\n\n철종 시기에 발생한 임술농민봉기는 삼정의 문란에 대한 반발...
98,고종 즉위 직후 흥선대원군이 비변사를 폐지한 목적은 무엇이었나요?,왕권 강화,왕권 강화 \n\n고종 즉위 직후 흥선대원군은 비변사를 폐지하고 의정부와 삼군부를...,채점 결과 : 4점\n\n흥선대원군이 비변사를 폐지한 목적이 왕권 강화라는 점을 정...


In [17]:
import re

# 정규식 패턴: '채점 결과\s*:\s*(\d+)점'
#   \s* : 공백문자 0개 이상
#   (\d+) : 하나 이상의 숫자를 그룹화(캡처)
#   뒤에 '점'이 따라옴
pattern = r'채점 결과\s*:\s*(\d+)점'

# df에 새 컬럼 'score_num' 추가
ori['score_num'] = ori['채점결과'].apply(
    lambda x: re.search(pattern, x).group(1) if re.search(pattern, x) else None
)

# 'score_num'을 정수형으로 변환
ori['score_num'] = ori['score_num'].astype(float)

ori


,질문,답변,LLM답변,채점결과,score_num
0,조선을 건국한 왕은 누구인가요?,태조,태조 이성계,채점 결과 : 4점\n출력값 : 1 \n\n이유 : '태조 이성계'는 '태조'를 포...,4.0
1,태조가 고려에 등 돌린 결정적 사건은 무엇인가요?,위화도 회군,,"⚠️ 2점\n\n정답인 '위화도 회군'을 언급하지 않았습니다. \n\n추가로, 해...",NaN
2,조선 3대 왕은 누구인가요?,태종,태종 이방원,"채점 결과 : 4점\n\n정답인 '태종'을 포함하면서, 추가 설명이 있으므로 4점을...",4.0
3,사병을 혁파한 왕은 누구인가요?,태종,태종 이방원,"채점 결과 : 4점\n\n정답인 '태종'을 포함하면서, 해당 인물의 출신과 업적을 ...",4.0
4,한양 천도를 주도한 왕은 누구인가요?,태조,정도전,"채점 결과 : 2점\n\n정답인 '태조'가 아닌 '정도전'으로 답변하였으므로, 점수...",2.0
...,...,...,...,...,...
95,홍경래의 난(1811)이 일어난 시기의 왕은 누구인가요?,순조,순조,채점 결과 : 5점 \n\n정답과 완전히 일치하는 답변입니다.,5.0
96,"1801년 신유박해 외에, 1815년에도 천주교 관련 옥사가 일어난 이유는 무엇인가요?",사교 탄압,신유박해 이후에도 천주교 신자들이 계속해서 발견되었기 때문입니다.,채점 결과 : 4점\n\n정답인 '사교 탄압'을 포함하면서도 추가적인 설명을 잘 해...,4.0
97,철종 시기에 발생한 임술농민봉기(1862)는 주로 무엇에 대한 반발이었나요?,과도한 세금,삼정의 문란에 대한 반발,채점 결과 : 4점\n\n철종 시기에 발생한 임술농민봉기는 삼정의 문란에 대한 반발...,4.0
98,고종 즉위 직후 흥선대원군이 비변사를 폐지한 목적은 무엇이었나요?,왕권 강화,왕권 강화 \n\n고종 즉위 직후 흥선대원군은 비변사를 폐지하고 의정부와 삼군부를...,채점 결과 : 4점\n\n흥선대원군이 비변사를 폐지한 목적이 왕권 강화라는 점을 정...,4.0
